<a href="https://colab.research.google.com/github/manaswitasolanki/leaf-disease-detection/blob/main/plant_disease_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PLANT DISEASE PREDICTION USING IMAGES OF HEALTHY AND UNHEALTHY LEAVES**


---

DATASET : '*plant_village*' from Github
         ( link:https://github.com/spMohanty/PlantVillage-Dataset)
This dataset has more than 50,000 images of plant leaves divided into 38 categories. 

To classify the images we will use a Convolution Neural Network.
(It basically contains layers just like an ANN but it uses techniques like applyling filters , max pooling ( reducing the input layer size) , flattening etc.) 


The PlantVillage dataset consists of 54303 healthy and unhealthy leaf images divided into 38 categories by species and disease.

First we will load the dataset from Github 


In [1]:
#importing the libraries we need to download the dataset
import zipfile
import os

In [2]:
#downloading dataset from github  repository we will save it in tmp folder
!wget --no-check-certificate \
    "https://github.com/spMohanty/PlantVillage-Dataset/archive/refs/heads/master.zip" \
    -O "/tmp/plant-village.zip"

--2022-08-03 18:12:01--  https://github.com/spMohanty/PlantVillage-Dataset/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/spMohanty/PlantVillage-Dataset/zip/refs/heads/master [following]
--2022-08-03 18:12:01--  https://codeload.github.com/spMohanty/PlantVillage-Dataset/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/plant-village.zip’

/tmp/plant-village.     [ <=>                ]   2.33G  33.2MB/s    in 82s     

2022-08-03 18:13:24 (29.2 MB/s) - ‘/tmp/plant-village.zip’ saved [2503721860]



In [3]:
zip_ref = zipfile.ZipFile('/tmp/plant-village.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

After discovering about ways of implementing CNN , I found Keras very interesting and easy :D but after several errors I realized that unlike other techniques, Keras only works if your input is in the form of a numpy array 
It took me a very long time to pre-process the data because of my beginner knowledge in python.

In [4]:
import cv2    #we will use it to preprocess the dataset
import numpy as np   #used for mathematical functions like matrices

We will set the height and width of the image to 128 . CNN works better if the size of the input images are the same (we can also apply padding but I felt that will use additional memory)
The number of channels are nothing but the depth of the image - I am using RGB which has a depth of 3 (greyscale has 1 etc.)

In [5]:
img_rows=128 #no.of rows in an image
img_cols=128  #no.of columns in an image
num_channel=3 # no. of channels (rgb)

Initially found it difficult to know where the files are located on colab but I soon discovered about the os library and now can directly select folders using paths

In [6]:
data_dir_list = os.listdir('/tmp/PlantVillage-Dataset-master/raw/color') #the path where the dataset is stored

As we know that the plant village dataset has 38 categories we will specify the number of classes as 38
We will use a dictionary to assign values to the different classes
We will create empty numpy arrays for the images and the labels

In [7]:
num_classes = 38
labels_name={'Apple___Apple_scab':0,
             'Apple___Black_rot':1,
             'Apple___Cedar_apple_rust':2,
             'Apple___healthy':3,
             'Blueberry___healthy':4,
             'Cherry_(including_sour)___Powdery_mildew':5,
             'Cherry_(including_sour)___healthy':6,
             'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot':7,
             'Corn_(maize)___Common_rust_':8,
             'Corn_(maize)___Northern_Leaf_Blight':9,
             'Corn_(maize)___healthy':10,
             'Grape___Black_rot':11,
             'Grape___Esca_(Black_Measles)':12,
             'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)':13,
             'Grape___healthy':14,
             'Orange___Haunglongbing_(Citrus_greening)':15,
             'Peach___Bacterial_spot':16,
             'Peach___healthy':17,
             'Pepper,_bell___Bacterial_spot':18,
             'Pepper,_bell___healthy':19,
             'Potato___Early_blight':20,
             'Potato___Late_blight':21,
             'Potato___healthy':22,
             'Raspberry___healthy':23,
             'Soybean___healthy':24,
             'Squash___Powdery_mildew':25,
             'Strawberry___Leaf_scorch':26,
             'Strawberry___healthy':27,
             'Tomato___Bacterial_spot':28,
             'Tomato___Early_blight':29,
             'Tomato___Late_blight':30,
             'Tomato___Leaf_Mold':31,
             'Tomato___Septoria_leaf_spot':32,
             'Tomato___Spider_mites Two-spotted_spider_mite':33,
             'Tomato___Target_Spot':34,
             'Tomato___Tomato_Yellow_Leaf_Curl_Virus':35,
             'Tomato___Tomato_mosaic_virus':36,
             'Tomato___healthy':37,
             
             }
img_data_list=[]  #an array containing all the images
labels_list = []  #an array containing all the labels/classes

We are creating a nested for loop , the first one loops through the entire directory(I have selected the color folder as it had all the rgb images) and selects the 38 different datasets .The label is also added using the dictionary we made earlier.
The second for loop runs through each label and reads each the image files using imread , resizes each image into the height and width we want , appends each image to the image array , the label is also appended to the label array

While running the code the limit of 12GB RAM surpassed and I had to reduce the number of images in each label to 250

In [8]:

		
for dataset in data_dir_list:
	img_list=os.listdir('/tmp/PlantVillage-Dataset-master/raw/color'+'/'+ dataset)
	print ('Loading the images of dataset-'+'{}\n'.format(dataset))
	label = labels_name[dataset]
	for img in img_list[:250]:
		input_img=cv2.imread('/tmp/PlantVillage-Dataset-master/raw/color' + '/'+ dataset + '/'+ img )
		input_img_resize=cv2.resize(input_img,(128,128))
		img_data_list.append(input_img_resize)
		labels_list.append(label)


Loading the images of dataset-Tomato___Early_blight

Loading the images of dataset-Raspberry___healthy

Loading the images of dataset-Tomato___Late_blight

Loading the images of dataset-Potato___healthy

Loading the images of dataset-Apple___healthy

Loading the images of dataset-Strawberry___Leaf_scorch

Loading the images of dataset-Peach___Bacterial_spot

Loading the images of dataset-Potato___Late_blight

Loading the images of dataset-Apple___Cedar_apple_rust

Loading the images of dataset-Peach___healthy

Loading the images of dataset-Tomato___Bacterial_spot

Loading the images of dataset-Tomato___Septoria_leaf_spot

Loading the images of dataset-Cherry_(including_sour)___Powdery_mildew

Loading the images of dataset-Blueberry___healthy

Loading the images of dataset-Cherry_(including_sour)___healthy

Loading the images of dataset-Grape___healthy

Loading the images of dataset-Corn_(maize)___healthy

Loading the images of dataset-Tomato___Tomato_Yellow_Leaf_Curl_Virus

Loading the

img_data now contains all the images in array form , we set the type of each element as float 

In [9]:
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')


Each value would be in a range of 0 to 255 which will take a lot of computation time so we will divide each value by 255 to make it in a range of 0 to 1 

In [10]:
img_data /= 255  #normalizing the data so that the value is less than 1 -fast computation

To check how many images are there

In [11]:
print (img_data.shape)  #total 9000 images ,it was 50000 earlier but ram stopped owking to reduced it

(9402, 128, 128, 3)


To check how many images we have in each label

In [13]:
labels = np.array(labels_list)
print(np.unique(labels,return_counts=True))  #shows that 250 images are taken from each class

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37]), array([250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250,
       250, 250, 250, 250, 250, 250, 250, 250, 250, 152, 250, 250, 250,
       250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250]))


We will use scikit learn to do a train test split
The test size is 0.2 which is 20 percent of the data ,80 percent will be used for training

In [14]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [15]:
Y = np_utils.to_categorical(labels, num_classes)
x,y = shuffle(img_data,Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

Checking the shapes as it is very important for the X_train and y_train to have the same examples .
We are using one hot encoding which basically means each element in the array will represent one label (38 elements here) and if the image belongs to a label the corresponding element is 1 the rest are 0s
We can see that by printing the first 5 elements of y_train

In [18]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(y_train[:5])



(7521, 128, 128, 3)
(1881, 128, 128, 3)
(7521, 38)
(1881, 38)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Now begins the the most important part 
To make the model using CNN we will first import the needed libraries 

In [20]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D


We will use 3 layers for our model 

Input layer : We will perform the convolution operation first by using 32 filters each of the size 3x3 .The activation function we use will be ReLu ,it is the best for normalizing data. Max pooling basically reduces the input size and here we will take a stride of 2 

Hideen layer: It is the same as input layer but it has more filters 

Output layer:We will flatten our output (make it 1D) and put it as an input to this layer .The final layer should have the same size as the number of classes .We will use softmax now as

In [25]:
#define the model
# input_shape=img_data[0].shape
# model = Sequential()

# model.add(Convolution2D(32, 3,3,border_mode='same',input_shape=input_shape))
# model.add(Activation('relu'))
# model.add(Convolution2D(32, 3, 3))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Convolution2D(64, 3, 3))
# model.add(Activation('relu'))
# #model.add(Convolution2D(64, 3, 3))
# #model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax'),
    
])



We will compile all the layers .Categorical crossentropy is used as we have more than two categories

In [26]:
cnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Checking out how the shapes of the input and output will be in each layer

In [27]:
cnn.summary()
cnn.get_config()
cnn.layers[0].get_config()
cnn.layers[0].input_shape			
cnn.layers[0].output_shape			
cnn.layers[0].get_weights()
np.shape(cnn.layers[0].get_weights()[0])
cnn.layers[0].trainable

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 57600)             0         
                                                                 
 dense_2 (Dense)             (None, 64)               

True

Finally training the model . It took around 30 mins for this to run.A higher batch size will take less time but the accuracy will be lower so went with 16

In [28]:
hist = cnn.fit(X_train, y_train, batch_size=16, epochs=10, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
471/471 [==============================] - 159s 337ms/step - loss: 2.3455 - accuracy: 0.3574 - val_loss: 1.5435 - val_accuracy: 0.5476
Epoch 2/10
471/471 [==============================] - 158s 336ms/step - loss: 1.0863 - accuracy: 0.6761 - val_loss: 1.1230 - val_accuracy: 0.6704
Epoch 3/10
471/471 [==============================] - 159s 337ms/step - loss: 0.5977 - accuracy: 0.8128 - val_loss: 1.0185 - val_accuracy: 0.7055
Epoch 4/10
471/471 [==============================] - 159s 337ms/step - loss: 0.2939 - accuracy: 0.9095 - val_loss: 1.1364 - val_accuracy: 0.7033
Epoch 5/10
471/471 [==============================] - 159s 337ms/step - loss: 0.1741 - accuracy: 0.9442 - val_loss: 1.1497 - val_accuracy: 0.7294
Epoch 6/10
471/471 [==============================] - 160s 339ms/step - loss: 0.1081 - accuracy: 0.9653 - val_loss: 1.4462 - val_accuracy: 0.7108
Epoch 7/10
471/471 [==============================] - 158s 336ms/step - loss: 0.0809 - accuracy: 0.9766 - val_loss: 1.5482 -

The average accuracy is 66 percent 

In [29]:
cnn.evaluate(X_test,y_test)

59/59 [==============================] - 9s 157ms/step - loss: 1.8508 - accuracy: 0.6683


[1.8508288860321045, 0.6682615876197815]

We will make the prediction and print the first 5 outputs 

In [30]:
y_pred = cnn.predict(X_test)
y_pred[:5]

array([[1.13544963e-19, 3.85957786e-13, 9.14536782e-21, 1.24451005e-11,
        8.23121684e-16, 6.59421866e-12, 1.15202736e-10, 8.20853004e-20,
        7.27910076e-27, 7.17235578e-32, 5.97415457e-30, 9.07640469e-08,
        3.99202962e-11, 1.72791526e-09, 2.13200972e-17, 3.05697689e-09,
        1.98940156e-10, 8.29694332e-13, 9.81666386e-01, 1.83332451e-02,
        6.55827203e-15, 2.59780836e-10, 2.00308575e-10, 5.70309986e-08,
        9.69793169e-12, 1.26097325e-18, 7.38118131e-14, 5.81349350e-17,
        1.16180474e-18, 4.70884284e-14, 1.86352977e-07, 2.43182496e-09,
        8.28678471e-18, 9.18475607e-09, 9.65010745e-15, 9.91380806e-13,
        2.89829343e-16, 5.35553383e-19],
       [2.30401126e-03, 2.53962384e-07, 3.45262725e-14, 5.68081362e-07,
        2.44635459e-08, 8.44080574e-17, 9.65469232e-11, 1.86137790e-17,
        4.97223097e-17, 2.93419730e-16, 1.01518608e-21, 7.56229389e-13,
        3.86219790e-06, 1.42044044e-07, 3.13509496e-11, 5.03266578e-17,
        1.88168451e-05,

The label with the highest value will be the predicted one, using argmax we get the following labels for the first 5 examples

In [33]:
y_classes = [np.argmax(element) for element in y_pred]
y_classes[:5] #the classes we get as prediction

[18, 31, 16, 6, 16]

These are the actual labels of the examples .These are difficult to view in one hot encoding 

In [32]:
y_test[:5]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]], dtype=float32)

4 out of 5 of the predictions are accurate :))

In [37]:
y_actual_classes=[np.argmax(element) for element in y_test]
print(y_actual_classes[:5])

[18, 32, 16, 6, 16]
